## Adam Babs
### FreeAgent Data Science - Internship
### Task solution

In [464]:
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
warnings.simplefilter('ignore')

In [465]:
cwd = os.getcwd()
df = pd.read_csv(os.path.join(cwd, 'attributes_report.csv'))


### Extracting data from the .log file

In [466]:
f = open(r'C:\Users\Adam Babs\Desktop\data_FreeAgent_internship\engagement_report.log', 'r')
results = [line[25:26] for line in f.readlines()]
f.close()

### Order check (making sure that companies numbers are in order in the .log file)

In [394]:
f = open(r'C:\Users\Adam Babs\Desktop\data_FreeAgent_internship\engagement_report.log', 'r')
order = [line[8:15] for line in f.readlines()]
f.close()


In [462]:
flag = 1
i = 1
while i < len(order): 
    if(order[i - 1] > order[i]): 
        flag = 0
    i += 1
      
if (flag==1) : 
    print ("Yes, List is sorted.") 
else : 
    print ("No, List is not sorted.") 

Yes, List is sorted.


### Calculating the number of weekly entries for each customer and adding the result to the dataframe

In [396]:
results = [int(i) for i in results]
weekly = [sum(results[x:x+7]) for x in range(0, len(results), 7)]


In [397]:
df['weekly entries']= weekly

In [451]:
df.head(3)

,company,company_type,subscribed_after_free_trial,weekly entries
0,346782,uk_limited_company,False,3
1,346783,uk_limited_company,False,0
2,346784,uk_limited_company,True,7


### Examining relations between features and finding imporant information

In [399]:
df.loc[df['weekly entries'] > 7]

,company,company_type,subscribed_after_free_trial,weekly entries
14,346796,uk_limited_company,True,11
34,346816,uk_limited_company,True,9
37,346819,uk_limited_company,True,11
45,346827,uk_limited_company,True,11
52,346834,uk_limited_company,True,12
...,...,...,...,...
826,347608,uk_sole_trader,True,10
827,347609,uk_sole_trader,True,10
839,347621,uk_sole_trader,True,9
864,347646,universal_company,True,11


In [454]:
len(df.loc[(df['weekly entries'] > 6) & (df['subscribed_after_free_trial'] == False)])

0

## In case there were more than 6 entries, all users paid for the subscription

In [401]:
# uk_limited_companies always bought the subscription above 6 entries a week
len(df.loc[(df['weekly entries'] ==6) & (df['subscribed_after_free_trial'] == True) & (df['company_type'] == 'uk_limited_company')])

14

In [402]:
# universal companies 
len(df.loc[(df['weekly entries'] ==6) & (df['subscribed_after_free_trial'] == True) & (df['company_type'] == 'universal_company')])

0

In [403]:

len(df.loc[(df['weekly entries'] ==6) & (df['subscribed_after_free_trial'] == True) & (df['company_type'] == 'uk_sole_trader')])

2

In [461]:
len(df.loc[(df['weekly entries'] < 6)])

803

In [460]:

len(df.loc[(df['weekly entries'] < 6) & (df['subscribed_after_free_trial'] == True) ])

20

## Model

In [404]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

from sklearn import model_selection, naive_bayes, svm
from sklearn.linear_model import LinearRegression

In [405]:
from sklearn.utils import shuffle
df_model = shuffle(df)

In [406]:
df_model['subscribed_after_free_trial'] = df_model['subscribed_after_free_trial'].apply(lambda x: 1 if x == True else 0)

In [407]:
X = df_model['weekly entries']
y = df_model['subscribed_after_free_trial']


In [408]:
X = np.array(X)
X = X.reshape(-1, 1)

In [409]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [425]:
logreg = LogisticRegression()
score_logreg1 = cross_val_score(logreg, X, y, cv=5)

print("Logistic Regression Classifier Accuracy: ")
print("5-fold cross validation average: ", score_logreg1.mean())
print("Highest accuracy: ", score_logreg1.max())

Logistic Regression Classifier Accuracy: 
5-fold cross validation average:  0.9726202067122429
Highest accuracy:  0.9842105263157894


In [426]:
linreg = LinearRegression()
score_linear = cross_val_score(linreg, X, y, cv=5)

print("Linear Regression Classifier Accuracy: ")
print("5-fold cross validation average: ", score_linear.mean())
print("Highest accuracy: ", score_linear.max())

Linear Regression Classifier Accuracy: 
5-fold cross validation average:  0.7373162280261513
Highest accuracy:  0.7668831901521169


In [429]:
SGD = linear_model.SGDClassifier()
a = cross_val_score(SGD, X, y, cv=5)

print("Stochastic Gradient Descent Classifier Accuracy: ")
print("5-fold cross validation average: ", a.mean())
print("Highest accuracy: ", a.max())

Stochastic Gradient Descent Classifier Accuracy: 
5-fold cross validation average:  0.9736894884097971
Highest accuracy:  0.9842105263157894


# 2nd model with matching the company type

In [430]:
from sklearn.utils import shuffle
df_model2 = shuffle(df)

In [431]:
df_model2['subscribed_after_free_trial'] = df_model2['subscribed_after_free_trial'].apply(lambda x: 1 if x == True else 0)


In [432]:
mymap = {'uk_limited_company':1, 'uk_sole_trader':2, 'universal_company':3}
df_model2 = df_model2.applymap(lambda s: mymap.get(s) if s in mymap else s)

In [433]:
df_model2.head()

,company,company_type,subscribed_after_free_trial,weekly entries
372,347154,1,0,0
580,347362,2,1,8
51,346833,1,0,1
443,347225,1,0,3
293,347075,1,0,1


In [441]:
X2 = df_model2[[ 'company_type', 'weekly entries']]
y2 = df_model2['subscribed_after_free_trial']


In [442]:
logreg = LogisticRegression()
score_logreg2 = cross_val_score(logreg, X2, y2, cv=5)

print("Logistic Regression Classifier Accuracy: ")
print("5-fold cross validation average: ", score_logreg2.mean())
print("Highest accuracy: ", score_logreg2.max())

Logistic Regression Classifier Accuracy: 
5-fold cross validation average:  0.9747476311488436
Highest accuracy:  0.9842105263157894


In [443]:
reg = LinearRegression()
score_linear2 = cross_val_score(reg, X2, y2, cv=5)

print("Linear Regression Classifier Accuracy: ")
print("5-fold cross validation average: ", score_linear2.mean())
print("Highest accuracy: ", score_linear2.max())

Linear Regression Classifier Accuracy: 
5-fold cross validation average:  0.7447233948888987
Highest accuracy:  0.7699231517743009


In [444]:
SGD = linear_model.SGDClassifier()
score_SGD2 = cross_val_score(SGD, X2, y2, cv=5)

print("Stochastic Gradient Descent Classifier Accuracy: ")
print("5-fold cross validation average: ", score_SGD2.mean())
print("Highest accuracy: ", score_SGD2.max())

Stochastic Gradient Descent Classifier Accuracy: 
5-fold cross validation average:  0.9737005690491498
Highest accuracy:  0.9842105263157894
